In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.sparse.linalg
import functools

# generates a random quntum state for n spins 1/2 as
# a rank n tensor of probability amplitudes psi_{0,1,2,\dots,n-1}
def random(n):
  x = np.random.rand(2**n)+1.j*np.random.rand(2**n)
  x = x/np.sqrt(np.vdot(x,x))
  x = np.reshape(x,list(np.repeat(2,n)))
  return x

# generates a random ground state of a classical 1D Ising model with n spins 1/2 as
# a rank n tensor of probability amplitudes psi_{0,1,2,\dots,n-1}
def Is_GS(n):
  x = np.random.rand(2)+1.j*np.random.rand(2)
  x = x/np.sqrt(np.vdot(x,x))
  v = np.zeros(2**n,dtype=np.complex128)
  v[0] = x[0]
  v[2**n-1] = x[1]
  v = np.reshape(v,list(np.repeat(2,n)))
  return v

#generates an expectation value O of an operator o acting at a site s 
#O=psi_{i0,...,i_{s-1},j,i_{s+1},...,i_n}^* o_{j,k} psi_{i0,...,i_{s-1},k,i_{s+1},...,i_n} (Einstein's convention implied) 
def EV_1s(psi,s,op):
   n = len(psi.shape)
   psiop = np.tensordot(op,psi,axes=((1,),(s,)))
   psiop = np.transpose(np.reshape(psiop,(2,2**(s),2**(n-s-1))),(1,0,2))
   EV = np.vdot(np.reshape(psi,-1),np.reshape(psiop,-1))
   return np.real(EV)

#generates an expectation value O of an operator o acting at sites s,s+1 
#O=psi_{i0,...,i_{s-1},j,k,i_{s+2},...,i_n}^* o_{j,k,l,m} psi_{i0,...,i_{s-1},l,m,i_{s+2},...,i_n} (Einstein's convention implied)
def EV_2s(psi,s,op):
   n = len(psi.shape)
   psiop = np.tensordot(op,psi,axes=((2,3),(s,s+1)))
   psiop = np.transpose(np.reshape(psiop,(4,2**(s),2**(n-s-2))),(1,0,2))
   EV = np.vdot(np.reshape(psi,-1),np.reshape(psiop,-1))
   return np.real(EV)
 


In [3]:
def to_left_canonical_mps(state):
    dim = len(state.shape)
    bond_dim = 1

    mps = []

    #reshape the state tensor to get 2x2 U matrix from SVD and perform first SVD
    state = np.reshape(state, (2,-1))
    U,S,V = np.linalg.svd(state, full_matrices=False)

    #U is 2x2 matrix, ready to use as first part of MPS, S is absorbed into V and will be further factorized
    mps.append(U)
    V = np.diag(S)@V

    # loop for rank 3 product tensors with number of left bonds smaller than right bonds
    for i in range(int((dim-1)/2)):
        bond_dim += 1

        V = np.reshape(V, (2**bond_dim,-1))
        U,S,V = np.linalg.svd(V, full_matrices=False)
        U = np.reshape(U, (2**(bond_dim-1),2,-1))  

        mps.append(U)
        V = np.diag(S)@V

    #cases of even dimension need to have more bonds at the beginning of the 2nd loop
    if(dim%2 == 0): bond_dim += 1

    #loop for the remaining rank 3 tensors
    for i in range(int(dim/2 - 1)):
        V = np.reshape(V, (2**(bond_dim),-1))
        U,S,V = np.linalg.svd(V, full_matrices=False)
        U = np.reshape(U, (2**(bond_dim-1),2,-1))

        mps.append(U)
        V = np.diag(S)@V

        bond_dim -= 1
    
    mps.append(V)
    return(mps)

def get_state(MPS):
    #contracting neighbouring indices of every tensor being a part of MPS
    state = np.tensordot(MPS[0], MPS[1], axes = [1,0])

    for i in range(2, len(MPS)):
        state = np.tensordot(state, MPS[i], axes = [-1,0])

    return state

def check_state(MPS,state):
    eps = 1e-15
    is_close = np.allclose(abs(get_state(MPS)), abs(state), atol = eps)
    return is_close

def bond_dim_check(mps):
    bond_dim = 2
    if (len(mps) < 2): 
        print("too small dimension of MPS!")
        return 0
    for i in range(len(mps)):
        a = mps[i].shape
        for j in range(len(a)):
            if (bond_dim < a[j]):
                bond_dim = a[j]
    return bond_dim


In [4]:
dim = 7
eps = 1e-15
Z = np.diag([1,-1])

state = random(dim)


MPS = to_left_canonical_mps(state)
for i in range(len(MPS)):
    print(MPS[i].shape)

(2, 2)
(2, 2, 4)
(4, 2, 8)
(8, 2, 8)
(8, 2, 4)
(4, 2, 2)
(2, 2)


In [5]:
def is_left_canonical(MPS):
    eps = 1e-14
    unitary_counter = 0
    dim = len(MPS)

    #leftmost contracted matrix should be of shape 2x2 and should be unitary
    if(np.allclose(MPS[0]@MPS[0].T.conj(), np.diag([1,1]), atol = eps)): unitary_counter += 1

    #double contracted tensors in the middle of MPS should also give unitary matrices
    for i in range(1,len(MPS)-1):
        m = MPS[i]
        mt = m.T.conj()
        mmt = np.einsum('ijk,lji->kl', m, mt)
        bonds = mmt.shape[0]
        if(np.allclose(mmt, np.diag(np.ones(bonds)), atol = eps)): unitary_counter += 1


    if(unitary_counter == dim - 1): 
        return True
    else:
        return False



In [6]:
def left_canonical_mps_norm(MPS):
    #contraction of leftmost MPS matrices
    norm = np.einsum('ij,ki->jk',MPS[0], MPS[0].T.conj())
    
    #loop for contracting first the conjugate element of MPS followed by contraction of original one 
    for i in range(1,len(MPS)-1):
        norm = np.tensordot(norm, MPS[i].T.conj(), axes=[1,2])
        norm = np.einsum('ijk, ikl->lj',norm,MPS[i])
        print(norm.shape)

    #final contraction of two remaining matrices
    norm = np.tensordot(norm, MPS[-1].T.conj(), axes=[1,1])
    norm = np.einsum('ij, ij',norm,MPS[-1])
    return np.real(norm)

def state_exp_value(MPS):
    #contraction of leftmost MPS matrices
    norm = np.einsum('ij,ki->jk',MPS[0], MPS[0].T.conj())

    #loop for contracting first the conjugate element of MPS followed by contraction of original one
    for i in range(1,len(MPS)-1):
        norm = np.tensordot(norm, MPS[i].T.conj(), axes=[1,2])
        norm = np.einsum('ijk, ikl->lj',norm,MPS[i])

    #contracting Z operator with rightmost matrices
    Z_contr = np.tensordot(MPS[-1], Z, axes=[1,0])
    Z_contr = np.tensordot(Z_contr, MPS[-1].T.conj(), axes=[1,0])
    
    #final double contraction loop result with result of operator Zcontraction 
    norm = np.einsum('ij, ij',norm,Z_contr)
    return np.real(norm)

In [7]:
def trunc_left_canonical_mps(state, isTruncated:bool):
    eps = 1e-15
    dim = len(state.shape)
    bond_dim = 1

    mps = []

    #reshape the state tensor to get 2x2 U matrix from SVD and perform first SVD
    state = np.reshape(state, (2,-1))
    U,S,V = np.linalg.svd(state, full_matrices=False)

    #U is 2x2 matrix, ready to use as first part of MPS, S is absorbed into V and will be further factorized
    mps.append(U)
    V = np.diag(S)@V

    # loop for rank 3 product tensors with number of left bonds smaller than right bonds
    for i in range(dim-2):
        bond_dim = U.shape[-1]

        V = np.reshape(V, (2*bond_dim,-1))
        U,S,V = np.linalg.svd(V, full_matrices=False)

        #truncation (discarding singular values and corresponding collumn vectors from U and row vectors from V) if singular value is smaller than eps = 1e-15
        if(isTruncated == True):
            j = 0
            while(j < len(S)):
                if (abs(S[j]) < eps):
                    U = np.delete(U, j, 1)
                    S = np.delete(S, j)
                    V = np.delete(V, j, 0)
                else:
                    j += 1
    
        U = np.reshape(U, (bond_dim,2,-1))
        mps.append(U)
        V = np.diag(S)@V



    
    mps.append(V)
    return(mps)

In [8]:
def to_mixed_canonical_mps(mps, site):
    dim = len(mps)
    MPS = mps.copy()

    for i in reversed(range(site+1,dim)):

        right_bond_shape= mps[i].shape
        left_bond_shape = mps[i-1].shape

        V = np.reshape(MPS[i], (left_bond_shape[-1], -1))
        U,S,V = np.linalg.svd(V, full_matrices=False )

        MPS[i] = np.reshape(V, right_bond_shape)
        U = MPS[i-1] @ U @ np.diag(S)
        MPS[i-1] = np.reshape(U, left_bond_shape)

    return MPS


In [9]:
def is_mixed_canonical(MPS, site):
    eps = 1e-14
    identity_counter = 0
    dim = len(MPS)
    

    #leftmost contracted matrix should be of shape 2x2 and should be unitary
    if(np.allclose(MPS[0]@MPS[0].T.conj(), np.diag([1,1]), atol = eps)): identity_counter += 1

    #double contracted tensors in the middle of MPS should contract to identity matrices
    for i in range(1,site):
        m = MPS[i]
        mt = m.T.conj()
        mmt = np.einsum('ijk,lji->kl', m, mt)
        bonds = mmt.shape[0]
        if(np.allclose(mmt, np.diag(np.ones(bonds)), atol = eps)): identity_counter += 1
        
    for i in range(site+1, dim-1):
        m = MPS[i]
        mt = m.T.conj()
        mmt = np.einsum('ijk,lji->kl', mt, m)
        bonds = mmt.shape[0]
        if(np.allclose(mmt, np.diag(np.ones(bonds)), atol = eps)): identity_counter += 1
  
    #check for rightmost matrix
    if(np.allclose(MPS[-1]@MPS[-1].T.conj(), np.diag([1,1]), atol = eps)): identity_counter += 1

    if(identity_counter == dim - 1): 
        return True
    else:
        return False


def mixed_canonical_mps_norm(mps, site):
    if(is_mixed_canonical(mps,site) == False ): return "not mixed canonical"

    #for mixed canonical norm we need to contract only orthogonality center
    m = mixed_mps[site]
    mt = m.T.conj()
    mmt = np.einsum('ijk,kji', mt, m)

    return np.real(mmt)


In [10]:
def one_site_exp_value(state, site, operator):
    #transforming state to mixed canonical mps with orthogonality center at site 'site'
    mps = trunc_left_canonical_mps(state, isTruncated=True)
    mps = to_mixed_canonical_mps(mps, site)
    
    m = mps[site]
    mt = m.T.conj()

#left and right matrices have only 2 indices whereas tensors in between have 3, hence 3 cases
    exp_val = 0
    if(site == 0): 
        m = np.einsum('ij,ik->jk', m, operator)
        exp_val = np.einsum('ij,ij', m, mt)
    elif(site == len(mps)-1):
        m = np.einsum('ij,jk->ik', m, operator)
        exp_val = np.einsum('ij,ji', m, mt)
    else:
        m = np.einsum('ijk,jl->ilk', m, operator)
        exp_val = np.einsum('ijk,kji', m, mt)


    return np.real(exp_val)


def two_site_exp_value(state, site, operator):
    if(site >= len(state.shape) - 1): return "Error: site too large"
    #transforming state to mixed canonical mps with orthogonality center at site 'site'
    mps = trunc_left_canonical_mps(state, isTruncated=True)
    mps = to_mixed_canonical_mps(mps, site)
    
    m = mps[site]
    mt = m.T.conj()
    m_next = mps[site+1]
    mt_next = m_next.T.conj()
    
    #left and right matrices have only 2 indices whereas tensors in between have 3, hence 4 cases
    exp_val = 0

    if(len(state.shape) == 2):
        m = np.einsum('ki,kj->ij', m, operator)
        m = np.einsum('ik,jk->ji', mt, m)
        m_next = np.einsum('ik,kj->ij', m_next, operator)
        m_next = np.einsum('ki,jk->ji', mt_next, m_next)

        exp_val = np.einsum('ij,ij', m, m_next)

    elif(site == 0): 
        m = np.einsum('ki,kj->ij', m, operator)
        m = np.einsum('ik,jk->ji', mt, m)       
        m_next = np.einsum('ijk,jq->iqk', m_next, operator)
        m_next = np.einsum('ijk,pjr->prki', mt_next, m_next)

        exp_val = np.einsum('ij,iljl', m, m_next)

    elif(site == len(mps)-2):
        m = np.einsum('ijk,jq->iqk', m, operator)
        m = np.einsum('ijk,pjr->prki', mt, m)
        m_next = np.einsum('ik,kj->ij', m_next, operator)
        m_next = np.einsum('ki,jk->ji', mt_next, m_next)       

        exp_val = np.einsum('ijil,jl', m, m_next)

    else:
        m = np.einsum('ijk,jq->iqk', m, operator)
        m = np.einsum('ijk,pjr->prki', mt, m)
        m_next = np.einsum('ijk,jq->iqk', m_next, operator)
        m_next = np.einsum('ijk,pjr->prki', mt_next, m_next)


        exp_val = np.einsum('ijik,jlkl', m, m_next)

    return np.real(exp_val)



In [11]:
import numpy as np
from scipy.sparse.linalg import LinearOperator
from scipy.sparse.linalg import eigsh


Z = np.diag([1,-1])
X = np.array([[0., 1.],[1., 0.]])

# generates a random quntum state for n spins 1/2 as
# a rank n tensor of probability amplitudes psi_{0,1,2,\dots,n-1}
def random(n):
  x = np.random.rand(2**n)+1.j*np.random.rand(2**n)
  x = x/np.sqrt(np.vdot(x,x))
  x = np.reshape(x,list(np.repeat(2,n)))
  return x

# generates a random ground state of a classical 1D Ising model with n spins 1/2 as
# a rank n tensor of probability amplitudes psi_{0,1,2,\dots,n-1}
def Is_GS(n):
  x = np.random.rand(2)+1.j*np.random.rand(2)
  x = x/np.sqrt(np.vdot(x,x))
  v = np.zeros(2**n,dtype=np.complex128)
  v[0] = x[0]
  v[2**n-1] = x[1]
  v = np.reshape(v,list(np.repeat(2,n)))
  return v

# apply one-dimensional quantum transverse Ising model  (QTIM) with open boundary conditions Hamiltonian
#H = -\sum_{i=0}^{n-2} Z_i Z_j - h \sum_{i=0}^{n-1} X_i - h_Z \sum_{i=0}^{n-1} Z_i.
# to a state given by  a rank n tensor of probability amplitudes psi_{0,1,2,\dots,n-1}
def apply_H(psi,n,h,hz):
  sv = np.shape(psi)
  #print(sv)
  psio = np.zeros(sv)
  #print(np.shape(psio))
  op2 = np.transpose(np.tensordot(-Z,Z,axes=((),())),(0,2,1,3))
  op = -h*X-hz*Z
  for i in range(n-1):
    psit = np.tensordot(op2,psi,axes=((2,3),(i,i+1)))
    psit = np.transpose(np.reshape(psit,(4,2**(i),2**(n-i-2))),(1,0,2))
    psit = np.reshape(psit,sv) 
    psio = psio+psit
  for i in range(n):
    psit = np.tensordot(op,psi,axes=((1,),(i,)))    
    psit = np.transpose(np.reshape(psit,(2,2**(i),2**(n-i-1))),(1,0,2)) 
    psit = np.reshape(psit,sv) 
    psio = psio+psit
  return psio

# the same as apply_H but psi_{0,1,2,\dots,n-1} reshaped into a vector (required for the ground state computation)
def apply_H_wrap(psi,n,h,hz):
  sv = [2 for i in range(n)]
  #print(sv)
  psir = np.reshape(psi,sv)
  psio = apply_H(psir,n,h,hz)
  return np.reshape(psio,-1)
  

# compute energy of  the quantum transverse Ising model  (QTIM) with open boundary conditions Hamiltonian
def en_H(psi,n,h, hz):
  psio = apply_H(psi,n,h,hz)
  EV = np.vdot(np.reshape(psi,-1),np.reshape(psio,-1))
  return np.real(EV)

# compute the ground state energy of  the quantum transverse Ising model  (QTIM) with open boundary conditions Hamiltonian
def en_GS(n,h,hz):
  apply_H_hand = lambda x : apply_H_wrap(x,n,h,hz)
  dmx = 2**n
  A = LinearOperator((dmx,dmx), matvec=apply_H_hand)
  evals_small, evecs_small = eigsh(A, 1, which='SA')
  return evals_small[0]


n=7 # the qubit number
h=1 # the transverse field
hz = 1 # the longitudinal fiels
psi = random(n) # the random state
en = en_H(psi,n,h,hz)
enGS = en_GS(n,h,hz)

print("energy of a random state is "+ str(en))
print("the ground state energy is " +str(enGS))



energy of a random state is -5.497442062733022
the ground state energy is -14.319303055471966


In [90]:
def hamiltonian_mpo(state, h, hz):
    # we consider hamiltonian of the form 
    # H = -Z_i*Z_{i+1} - h*X_i - hz*Z_i
    # summed over all possible i's 

    # hamiltonian matrices
    Z = np.diag([1,-1])
    X = np.array([[0,1],[1,0]])
    minus_hZ = np.diag([-hz,-hz]) @ Z
    minus_hX = np.diag([-h,-h]) @ X

    # leftmost rank 3 tensor has the form [-h*X - hz*Z,    -Z,     1]
    H_0 = np.zeros((3,2,2))
    H_0[0,:,:] = minus_hX + minus_hZ
    H_0[1,:,:] = -1*Z
    H_0[2,:,:] = np.identity(2)

    # rank 4 tensors in the middle have the following form: 
    # [1,           0,      0]
    # [Z,           0,      0]
    # [-h*X-hz*Z,   -Z,     1]
    
    H_j = np.zeros((3,3,2,2))
    H_j[0,0,:,:] = np.identity(2)
    H_j[0,1,:,:] = np.zeros((2,2))
    H_j[0,2,:,:] = np.zeros((2,2))
    H_j[1,0,:,:] = Z
    H_j[1,1,:,:] = np.zeros((2,2))
    H_j[1,2,:,:] = np.zeros((2,2))
    H_j[2,0,:,:] = minus_hX + minus_hZ
    H_j[2,1,:,:] = -1*Z
    H_j[2,2,:,:] = np.identity(2)

    # rightmost rank 3 tensor has the form [1,   Z,   -h*X - hz*Z]
    H_n = np.zeros((3,2,2))
    H_n[0,:,:] = np.identity(2)
    H_n[1,:,:] = Z
    H_n[2,:,:] = minus_hX + minus_hZ

    # hamiltonian consists of number of tensors equal to its dimension
    hamiltonian = [H_0]
    
    dim = len(state.shape)

    for j in range(dim-2):
        hamiltonian.append(H_j)

    hamiltonian.append(H_n)


    return hamiltonian


def single_left_hamiltonian_contraction(left_tensor, mps, mpo, site):
    left_tensor = np.einsum("ijk, ipq -> qpjk", left_tensor, mps[site])
    left_tensor = np.einsum("ijkl, kpjq -> ipql", left_tensor, mpo[site])
    left_tensor = np.einsum("ijkl, pkl -> ijp", left_tensor, mps[site].T.conj())

    return left_tensor
        

def single_right_hamiltonian_contraction(right_tensor, mps, mpo, site):
    right_tensor = np.einsum("ijk, pqi -> pqjk", right_tensor, mps[site])
    right_tensor = np.einsum("ijkl, pkjq -> ipql", right_tensor, mpo[site])
    right_tensor = np.einsum("ijkl, lkp -> ijp", right_tensor, mps[site].T.conj())

    return right_tensor


def left_hamiltonian_contraction(mps, mpo, site):
    L = np.tensordot(mps[0], mpo[0], axes = [-2,-2])
    L = np.tensordot(L, mps[0].T.conj(), axes = (-1,-1))
    
    for i in range(1, site):
        L = single_left_hamiltonian_contraction(L, mps, mpo, i)    

    return L



def right_hamiltonian_contraction(mps, mpo, site):
    R = np.tensordot(mps[-1], mpo[-1], axes = (-1,-2))
    R = np.tensordot(R, mps[-1].T.conj(), axes = (-1,-2))
    # R = np.einsum("ij, kjl -> ikl", mps[-1], mpo[-1])
    # R = np.einsum("ijk, kl -> ijl", R, mps[-1].T.conj())

    dim = len(mps)
    for i in reversed(range(site+1, dim-1)):
        R = single_right_hamiltonian_contraction(R, mps, mpo, i)    

    return R


def hamiltonian_avg(state, mpo):
    site = int(len(state.shape)/2)
    
    # transforming state to mixed canonical mps
    mps = to_left_canonical_mps(state)
    mps = to_mixed_canonical_mps(mps, site)
    
    # performing contractions for left- and right-hand side of system
    # with orthogonality center contracted with R
    L = left_hamiltonian_contraction(mps, mpo, site)
    R = right_hamiltonian_contraction(mps, mpo, site-1)

    hamiltonian_exp = np.einsum('ijk, ijk', L, R)

    return np.real(hamiltonian_exp)

def hamiltonian_mps_avg(mps, mpo, site):
    # performing contractions for left- and right-hand side of system
    # with orthogonality center contracted with R
    if site == 0 or site == len(mps)-1:
        L = left_hamiltonian_contraction(mps, mpo, site)
        R = right_hamiltonian_contraction(mps, mpo, site)
    else:
        L = left_hamiltonian_contraction(mps, mpo, site)
        R = right_hamiltonian_contraction(mps, mpo, site-1)

    hamiltonian_exp = np.einsum('ijk, ijk', L, R)

    return np.real(hamiltonian_exp)





In [92]:
# check calculated energy of state vs energy calculated using reference function
state = random(6)
H = hamiltonian_mpo(state, 1, 1)

print("Obtained energy of the state:\t", hamiltonian_avg(state, H))
print("True reference energy:\t\t", en_H(state, len(state.shape),1,1))


Obtained energy of the state:	 -4.4279925667773465
True reference energy:		 -4.427992566777348


In [178]:
def to_DMRG_operator(mps, mpo, site):

    # assuming mixed canonical mps with orthogonality center at designed site
    dim = len(mps)
    
    # performing contractions for left- and right-hand side of system
    L = left_hamiltonian_contraction(mps, mpo, site)
    R = right_hamiltonian_contraction(mps, mpo, site)

    if(site == 0):
        DMRG_operator = np.einsum('ijk, lim -> ljmk', mpo[site], R)
        # DMRG_operator = np.einsum('mk, ljmk -> lj', mps[site].T.conj(), DMRG_operator)

    elif(site == dim-1):
        DMRG_operator = np.einsum('ijk, jlm -> ilkm', L, mpo[site])
        # DMRG_operator = np.einsum('ilkm, mk -> il', DMRG_operator, mps[site].T.conj())

    else:
        DMRG_operator = np.einsum('ijk, jlmn -> imlnk', L, mpo[site])
        # DMRG_operator = np.einsum('imlnk, pnk -> imlp', DMRG_operator, mps[site].T.conj())
        # DMRG_operator = np.einsum('imlp, olp -> imo', DMRG_operator, R)
        DMRG_operator = np.einsum('ijklm, pkq -> ijpqlm', DMRG_operator, R)

    return DMRG_operator


def apply_DMRG(DMRG_operator, tensor):
    pass


def minimize_energy_site(mps, Hamiltonian_MPO, site):
    # mps = MPS.copy()

    # comupte operator used for minimizing energy of the state at particular site
    DRMG_operator = to_DMRG_operator(mps, Hamiltonian_MPO, site)

    # determine shape for energy minimization DRMG operator (product of the first half of DRMG tensor dimensions)
    dimA = functools.reduce(lambda x,y: x*y, DRMG_operator.shape[:len(DRMG_operator.shape)//2])

    # create operator, whose eigenvector will be updated tensor at desired site
    A = np.reshape(DRMG_operator, (dimA, dimA))
    apply = lambda x: A @ x
    operator = scipy.sparse.linalg.LinearOperator((dimA, dimA), matvec = apply)

    # find smallest eigenvalue and corresponding eigenvector
    E, x = scipy.sparse.linalg.eigsh(operator, 1, which = 'SA')

    # reshape eigenvector to restore previous MPS tensor shape and substitude for updated component of MPS
    x = np.reshape(x, mps[site].shape)
    mps[site] = x

    return mps


def minimize_energy_site_explicit(MPS, Hamiltonian_MPO, site):
    mps = MPS.copy()

    # comupte operator used for minimizing energy of the state at particular site
    DRMG_operator = to_DMRG_operator(mps, Hamiltonian_MPO, site)

    # determine shape for energy minimization DRMG operator
    dimA = functools.reduce(lambda x,y: x*y, DRMG_operator.shape[:len(DRMG_operator.shape)//2])

    # A = np.reshape(MPS[site], (dimA))
    h = np.reshape(DRMG_operator, (dimA, dimA))

    # find smallest eigenvalue and corresponding eigenvector
    E, A = scipy.sparse.linalg.eigsh(h, 1, which = 'SA')

    # reshape eigenvector and substitude for i-th component of MPS
    A = np.reshape(A, mps[site].shape)
    mps[site] = A

    return mps


def minimize_energy_total(mps, mpo, num_sweeps):
    dim = len(mps)
    for sweeps in range(num_sweeps):
        # energy minimization from left to right
        for i in range(dim):
            mps = minimize_energy_site(mps, mpo, i)

        # energy minimization from right to left
        for i in reversed(range(dim)):
            mps = minimize_energy_site(mps, mpo, i)        
    
    return mps

In [179]:
#  działa?
site = 1
state = random(7)
H = hamiltonian_mpo(state, 1, 1)

# transform random quantum state into Matrix Product State
mps = to_left_canonical_mps(state)
mps = to_mixed_canonical_mps(mps, site)

print("\nBefore minimization:")
print(hamiltonian_mps_avg(mps, H, site))

# comparing two methods of minimizing energy: explicit construction of operator and using LinearOperator
print("\nAfter minimization:")
explicit = minimize_energy_site_explicit(mps, H, site)
print(hamiltonian_mps_avg(explicit, H, site))

print(hamiltonian_mps_avg(operator, H, site))
operator = minimize_energy_site(mps, H, site)


en = en_H(state,len(state.shape),1,1)
enGS = en_GS(len(state.shape),1,1)

print("\nReference energies")
print("Original:")
print(en)
print("\nGround state:")
print(enGS)

sweeps = 12
print("\nMPS energy after %d sweeps:" % sweeps)
mps = minimize_energy_total(mps, H, sweeps)
print(hamiltonian_mps_avg(mps, H, 1))



Before minimization:
-5.471712704835466

After minimization:
-7.9630226771021135
-10.304024385396419

Reference energies
Original:
-5.4717127048354675

Ground state:
-14.319303055471966

MPS energy after 12 sweeps:
-7.864332200564127
